## downloading necessary packages and setup

In [ ]:
%pip install ultralytics
%pip install pickle
%nvidia-smi    #for working on GPU
%pip install torch
%pip install tensorflow
import os
HOME = os.getcwd()
print(HOME)

## Fine-tuning YOLOV9

### Download the vehicle orientation dataset

In [3]:

import requests
import zipfile
import io
import os

HOME = os.getcwd()
print(HOME)
# creating a directory for the dataset
extraction_path = os.path.join(HOME, "Object_Detection_Dataset")
os.makedirs(extraction_path)

# download the dataset zip folder
url = 'https://sekilab-students.s3.ap-northeast-1.amazonaws.com/2021/vehicle-orientation-dataset/vehicle-orientation-5.zip'

response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Open the content of the response as a byte stream
    zip_content = io.BytesIO(response.content)
    
    # Use the zipfile library to extract the content
    with zipfile.ZipFile(zip_content, 'r') as zip_ref:
        # Extract all the contents into the specified directory
        zip_ref.extractall(extraction_path)
    print(f"Download and extraction successful! Files are extracted to {extraction_path}")
else:
    print("Failed to download the file. Status code:", response.status_code)



c:\Users\Nada\yoloproject\accident prediction\Github project
Download and extraction successful! Files are extracted to c:\Users\Nada\yoloproject\accident prediction\Github project\dataset


### Important step

In [ ]:
print("Please note: The directories for the training, testing, and validation image folders specified in the 'data.yaml' file must be MANUALLY UPDATED before running the following cells to reflect the full path where the files are located.")
print("For the training images, set the path as follows: " + f'{HOME}\\Object_Detection_Dataset\\vehicle-orientation-5\\train\\images')
print("For the validation images, set the path as follows: " + f'{HOME}\\Object_Detection_Dataset\\vehicle-orientation-5\\validate\\images')
print("For the testing images, set the path as follows: " + f'{HOME}\\Object_Detection_Dataset\\vehicle-orientation-5\\test\\images')


### Load the pretrained yolov9 model and further train it on the dataset

In [4]:
from ultralytics import YOLO

model = YOLO('yolov9e.pt')  # load an official model
dataset = f'{HOME}\\Object_Detection_Dataset\\vehicle-orientation-5\\data.yaml'
results = model.train(data=dataset ,device=0, epochs=10,batch=8 ) # further train the model

c:\Users\Nada\yoloproject\accident prediction\Github project


### Evaluate the yolov9 performance on the dataset

In [ ]:

model = YOLO('/runs/detect/train/weights/best.pt')
metrics = model.val(data=dataset)  
print(metrics)
metrics.box.map    # map50-95
metrics.box.map50  # map50
metrics.box.map75  # map75
metrics.box.maps   # a list contains map50-95 of each category

## Download the CCD dataset

In [8]:
import requests
import zipfile
import io
import os

HOME = os.getcwd()
print(HOME)
# creating a directory for the dataset
extraction_path = os.path.join(HOME, "Accident_Prediction_Dataset\\accidents")
os.makedirs(extraction_path)

# download the accident videos dataset zip folder
url = 'https://drive.google.com/uc?export=download&id=1fmcwGhr8JT9YfLUrlcvuCZ3ychi2eyFP'

response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Open the content of the response as a byte stream
    zip_content = io.BytesIO(response.content)
    
    # Use the zipfile library to extract the content
    with zipfile.ZipFile(zip_content, 'r') as zip_ref:
        # Extract all the contents into the specified directory
        zip_ref.extractall(extraction_path)
    print(f"The Accident videos are downloaded and extracted successfully in {extraction_path}")
else:
    print("Failed to download accident videos. Status code:", response.status_code)



extraction_path = os.path.join(HOME, "Accident_Prediction_Dataset\\Non_accidents")
os.makedirs(extraction_path)

# download the non-accident videos dataset zip folder
url = 'https://drive.google.com/uc?export=download&id=11ErpWQmmV5au2JOQVxwYl3ebtuugdlan'

response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Open the content of the response as a byte stream
    zip_content = io.BytesIO(response.content)
    
    # Use the zipfile library to extract the content
    with zipfile.ZipFile(zip_content, 'r') as zip_ref:
        # Extract all the contents into the specified directory
        zip_ref.extractall(extraction_path)
    print(f"The Non-accident videos are downloaded and extracted successfully in {extraction_path}")
else:
    print("Failed to download non-accident videos. Status code:", response.status_code)    


c:\Users\Nada\yoloproject\accident prediction\Github project\CCD_Normal_Sequences\train
c:\Users\Nada\yoloproject\accident prediction\Github project\CCD_Normal_Sequences\train\Accident_Prediction_Dataset\accidents


## Data Preprocessing

### Process accident videos 

In [ ]:
import os
import cv2
import math
from collections import defaultdict
import numpy as np
from ultralytics import YOLO
import tensorflow as tf
from pathlib import Path
from collections import defaultdict


#make sure that we are in the main project directory
print(os.getcwd())   

dataset_folder = os.path.join(HOME,'Accident_Prediction_Dataset\\accidents')

max_frames=50
max_objects=30
no_of_features=8

model_path = os.path.join(HOME,'runs\\detect\\train\\weights\\best.pt')
# loop over the videos in the dataset folder 
for filename in os.listdir(dataset_folder):
    # load the yolo model
    model = YOLO(model_path) 

    if filename.endswith('.mp4'):
        video_path = os.path.join(dataset_folder, filename)
        print('opening a new video')
        # Initialize a list to hold the sequences of frames
        sequences = []       
        max_i = 0
        # Store the track history
        track_history = defaultdict(lambda: [])
    
        # Open the video file
        cap = cv2.VideoCapture(video_path)

        # keep track of track_ids
        prev_track_ids = None
        # Loop through the video frames
        while cap.isOpened():
            # Read a frame from the video
            success, frame = cap.read()
    
            if success:
                # Initialize a list to hold the detections of the current frame 
                current_sequence = [None]* max_objects
    
                # Run YOLOv9 tracking on the frame, persisting tracks between frames
                results = model.track(frame, persist=True)

                # extract info/features from the results of yolov9 detection 
                if results is not None and results[0].boxes is not None :

                    #if the tracker failed to identify objects using ID, assume the track_ids of the previous frame 
                    if(results[0].boxes.id is not None):
                        track_ids = results[0].boxes.id.int().cpu().tolist()
                        prev_track_ids = track_ids
                    elif(prev_track_ids is None):   # no track_ids from previous frames 
                        track_ids = [0]*len(results[0].boxes)
                    else:
                        track_ids = prev_track_ids
                    boxes = results[0].boxes.xywh.cpu().tolist()
                    confidences = results[0].boxes.conf.tolist()
                    classes = results[0].boxes.cls.tolist()
                    for box, track_id, conf, classID in zip(boxes, track_ids, confidences, classes):
                        # extract the type and the orientation from the class_id
                        if(int(classID)==1):
                            class_id = 1
                            orientation = 1
                        elif(int(classID)==2):
                            class_id = 1
                            orientation = 2
                    
                        elif(int(classID)==3):
                            class_id = 1
                            orientation = 3
                            
                        elif(int(classID)==4):
                            class_id = 2
                            orientation = 1
                            
                        elif(int(classID)==5):
                            class_id = 2
                            orientation = 2
                            
                        elif(int(classID)==6):
                            class_id = 2
                            orientation = 3
                            
                        elif(int(classID)==7):
                            class_id = 3
                            orientation = 1
                            
                        elif(int(classID)==8):
                            class_id = 3
                            orientation = 2
                            
                        elif(int(classID)==9):
                            class_id = 3
                            orientation = 3
                            
                        elif(int(classID)==10):
                            class_id = 4
                            orientation = 1
                            
                        elif(int(classID)==11):
                            class_id = 4
                            orientation = 2
                            
                        elif(int(classID)==12):
                            class_id = 4
                            orientation = 3
                            
                        elif(int(classID)==13):
                            class_id = 5
                            orientation = 1
                            
                        elif(int(classID)==14):
                            class_id = 5
                            orientation = 2
                            
                        else:#int(classID)==15
                            class_id = 5
                            orientation = 3
                           
                        x, y, w, h = box   
                        conf = float (int (conf * 1000) / 1000)

                        # calculate the distance moved by every object between 2 consecutive frames 
                        track = track_history[track_id]
                        distance_moved = 0
                        if(len(track)!=0):
                            prev_position = track[-1]
                            distance_moved = math.sqrt( math.pow(float(x)-prev_position[0] , 2) + math.pow(float(y)-prev_position[1] , 2) )

                        track.append((float(x), float(y)))  # x, y center point  
                        
                        # add the object info to the list , keep consistency in sequences by TRACK_id
                        if(track_id<30):
                            current_sequence[track_id] = [ x, y, w, h, distance_moved, int(class_id) , int(orientation), conf]


                # replace the None values with list of zeros for padding 
                for i in range(len(current_sequence)):
                    if(current_sequence[i] is None):
                        current_sequence[i] = [0]*no_of_features  

                sequences.append(current_sequence)  
         
            else:
                break
    
        # Release the video capture object and close the display window
        cap.release()
        cv2.destroyAllWindows()

         # Pad the video to have a maximum of 50 frames
        while len(sequences) < max_frames:
            sequences.append([[0]*no_of_features]*max_objects)
        if len(sequences) > max_frames:
            sequences = sequences[:max_frames]

        sequences = np.array(sequences)
        np.save(os.path.join(HOME,'CCD_Accident_Sequences\\train\\')+Path(filename).stem+".npy", sequences)
        

### Process Non_accident videos

In [ ]:
import os
import cv2
import math
from collections import defaultdict
import numpy as np
from ultralytics import YOLO
import tensorflow as tf
from pathlib import Path
from collections import defaultdict


#make sure that we are in the main project directory
print(os.getcwd())   

dataset_folder = os.path.join(HOME,'Accident_Prediction_Dataset\\Non_accidents')

max_frames=50
max_objects=30
no_of_features=8

model_path = os.path.join(HOME,'runs\\detect\\train\\weights\\best.pt')
# loop over the videos in the dataset folder 
for filename in os.listdir(dataset_folder):
    # load the yolo model
    model = YOLO(model_path) 

    if filename.endswith('.mp4'):
        video_path = os.path.join(dataset_folder, filename)
        print('opening a new video')
        # Initialize a list to hold the sequences of frames
        sequences = []       

        # Store the track history
        track_history = defaultdict(lambda: [])
    
        # Open the video file
        cap = cv2.VideoCapture(video_path)

        # keep track of track_ids
        prev_track_ids = None
        # Loop through the video frames
        while cap.isOpened():
            # Read a frame from the video
            success, frame = cap.read()
    
            if success:
                # Initialize a list to hold the detections of the current frame 
                current_sequence = [None]* max_objects
    
                # Run YOLOv9 tracking on the frame, persisting tracks between frames
                results = model.track(frame, persist=True)

                # extract info/features from the results of yolov9 detection 
                if results is not None and results[0].boxes is not None :

                    #if the tracker failed to identify objects using ID, assume the track_ids of the previous frame 
                    if(results[0].boxes.id is not None):
                        track_ids = results[0].boxes.id.int().cpu().tolist()
                        prev_track_ids = track_ids
                    elif(prev_track_ids is None):   # no track_ids from previous frames 
                        track_ids = [0]*len(results[0].boxes)
                    else:
                        track_ids = prev_track_ids
                    boxes = results[0].boxes.xywh.cpu().tolist()
                    confidences = results[0].boxes.conf.tolist()
                    classes = results[0].boxes.cls.tolist()
                    for box, track_id, conf, classID in zip(boxes, track_ids, confidences, classes):
                        # extract the type and the orientation from the class_id
                        if(int(classID)==1):
                            class_id = 1
                            orientation = 1
                        elif(int(classID)==2):
                            class_id = 1
                            orientation = 2
                    
                        elif(int(classID)==3):
                            class_id = 1
                            orientation = 3
                            
                        elif(int(classID)==4):
                            class_id = 2
                            orientation = 1
                            
                        elif(int(classID)==5):
                            class_id = 2
                            orientation = 2
                            
                        elif(int(classID)==6):
                            class_id = 2
                            orientation = 3
                            
                        elif(int(classID)==7):
                            class_id = 3
                            orientation = 1
                            
                        elif(int(classID)==8):
                            class_id = 3
                            orientation = 2
                            
                        elif(int(classID)==9):
                            class_id = 3
                            orientation = 3
                            
                        elif(int(classID)==10):
                            class_id = 4
                            orientation = 1
                            
                        elif(int(classID)==11):
                            class_id = 4
                            orientation = 2
                            
                        elif(int(classID)==12):
                            class_id = 4
                            orientation = 3
                            
                        elif(int(classID)==13):
                            class_id = 5
                            orientation = 1
                            
                        elif(int(classID)==14):
                            class_id = 5
                            orientation = 2
                            
                        else:#int(classID)==15
                            class_id = 5
                            orientation = 3
                           
                        x, y, w, h = box   
                        conf = float (int (conf * 1000) / 1000)

                        # calculate the distance moved by every object between 2 consecutive frames 
                        track = track_history[track_id]
                        distance_moved = 0
                        if(len(track)!=0):
                            prev_position = track[-1]
                            distance_moved = math.sqrt( math.pow(float(x)-prev_position[0] , 2) + math.pow(float(y)-prev_position[1] , 2) )

                        track.append((float(x), float(y)))  # x, y center point  
                        
                        # add the object info to the list , keep consistency in sequences by TRACK_id
                        if(track_id<max_objects):
                            current_sequence[track_id] = [ x, y, w, h, distance_moved, int(class_id) , int(orientation), conf]


                # replace the None values with list of zeros for padding 
                for i in range(len(current_sequence)):
                    if(current_sequence[i] is None):
                        current_sequence[i] = [0]*no_of_features  

                sequences.append(current_sequence)  
         
            else:
                break
    
        # Release the video capture object and close the display window
        cap.release()
        cv2.destroyAllWindows()

         # Pad the video to have a maximum of 50 frames
        while len(sequences) < max_frames:
            sequences.append([[0]*no_of_features]*max_objects)
        if len(sequences) > max_frames:
            sequences = sequences[:max_frames]

        sequences = np.array(sequences)
        np.save(os.path.join(HOME,'CCD_Normal_Sequences\\train\\')+Path(filename).stem+".npy", sequences)
        
        

### Split the processed sequences into training and validation sets

In [6]:
import os
import shutil


# split the non_accident videos into 2 sets( train and validate )
source_dir = f'{HOME}\\CCD_Normal_Sequences\\train'
target_dir = f'{HOME}\\CCD_Normal_Sequences\\validate'

os.makedirs(target_dir)

files = os.listdir(source_dir)
# for non accident videos: training set = 2400 and validation set = 600
for file_name in files[:600]:
    shutil.move(os.path.join(source_dir, file_name), target_dir)


# split the accident videos into 2 sets( train and validate )
source_dir = f'{HOME}\\CCD_Accident_Sequences\\train'
target_dir = f'{HOME}\\CCD_Accident_Sequences\\validate'

os.makedirs(target_dir)

files = os.listdir(source_dir)
# for accident videos: training set = 800 and validation set = 200
for file_name in files[:200]:
    shutil.move(os.path.join(source_dir, file_name), target_dir)    


c:\Users\Nada\yoloproject\accident prediction\Github project\CCD_Normal_Sequences\train


## The LSTM model

### build and train the LSTM model on the processed sequences

In [ ]:
import os
import cv2
import numpy as np
from tensorflow.keras import regularizers
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras.callbacks import EarlyStopping 
from tensorflow.keras.layers import LSTM, Dense , Dropout,Masking
import pickle


# ===================== # customized functions for the evaluation matrices

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

# a function for loading the sequences 
def load_data(data_dir1 , data_dir2):
    x_train =[]
    y_train =[]
    # Loop through text files in data_dir
    for filename in os.listdir(data_dir1):
    # Read features from text file
        data = np.load(data_dir1+"/"+filename)
        x_train.append(data)
        y_train.append(1.0)
        
    for filename in os.listdir(data_dir2):
    # Read features from text file
        data = np.load(data_dir2+"/"+filename)
        x_train.append(data)
        y_train.append(0.0)  
        
    return x_train, y_train


# ===================== # the LSTM model

# size parameters
max_frames=50
max_objects=30
no_of_features=8

# Define model parameters
num_lstm_units = 128
num_dense_units = 64
dropout_rate = 0.3

# load the data for training
x_train , y_train =  load_data(f'{HOME}\\CCD_Accident_Sequences\\train',f'{HOME}\\CCD_Normal_Sequences\\train')     # read sequences from DESK
x_train = np.array(x_train)
y_train = np.array(y_train)
x_train = x_train.reshape(3200, -1, max_objects*no_of_features) # flatten each frame to 1D

# validation data
x_val ,y_val =  load_data( f'{HOME}\\CCD_Accident_Sequences\\validate',f'{HOME}\\CCD_Normal_Sequences\\validate'  )     # read sequences from DESK
x_val = np.array(x_val)
y_val = np.array(y_val)
# write the number of validation examples available
x_val = x_val.reshape(800, -1, max_objects*no_of_features) # flatten each frame to 1D

# Define the model
model1 = tf.keras.Sequential()

model1.add(Masking( mask_value=0 , input_shape=( max_frames, max_objects*no_of_features)))

model1.add(LSTM(num_lstm_units))
model1.add(Dropout(dropout_rate))

# Dense layer for prediction
model1.add(Dense(num_dense_units, activation='relu' ,kernel_regularizer=regularizers.l1_l2(l1=0.01, l2=0.01) ) )
model1.add(Dropout(dropout_rate))

# Output layer with sigmoid activation for probability
model1.add(Dense(1, activation='sigmoid' , kernel_regularizer=regularizers.l1_l2(l1=0.01, l2=0.01) ))
    
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)

# Compile the model
model1.compile(loss='binary_crossentropy', optimizer= optimizer, metrics=['accuracy',f1_m,precision_m,recall_m])

early_stopping = EarlyStopping(monitor='val_accuracy', patience=10)

history = model1.fit(x_train, y_train, epochs=16, batch_size=16, validation_data=(x_val, y_val), callbacks = early_stopping )

# Save the model after training
model1.save('accident_prediction_LSTM_model.keras') 

with open('historyLSTM.pickle', 'wb') as f:
    pickle.dump(history.history, f)



### plot the evaluation results

In [ ]:
import pickle
import matplotlib.pyplot as plt

# Load the history
with open('historyLSTM.pickle', 'rb') as f:
    history = pickle.load(f)

# Plot training & validation accuracy values
plt.plot(history['accuracy'])
plt.plot(history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.ylim(0.4, 0.9)
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()


# Access the values of custom metrics
recall_values = history['recall_m']
precision_values = history['precision_m']
f1_values = history['f1_m']

# Plotting

plt.plot(recall_values, label='Recall')
plt.title('Recall Over Epochs')
plt.ylabel('Recall')
plt.xlabel('Epoch')
plt.ylim(10,50)
plt.legend()
plt.show()

plt.plot(precision_values, label='Precision')
plt.title('Precision over Epochs')
plt.ylabel('precision')
plt.xlabel('Epoch')
plt.ylim(10,50)
plt.legend()
plt.show()

plt.plot(f1_values, label='F1 Score')
plt.title('F1 Score over Epochs')
plt.ylabel('F1_score')
plt.xlabel('Epoch')
plt.ylim(10, 50)
plt.legend()
plt.show()



## The GRU model

### build and train the GRU model on the processed sequences 

In [ ]:
import os
import cv2
import numpy as np
from tensorflow.keras import regularizers
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras.callbacks import EarlyStopping 
from tensorflow.keras.layers import GRU, Dense , Dropout ,Masking
import pickle

# ===================== # customized functions for the evaluation matrices

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

# a function for loading the sequences 
def load_data(data_dir1 , data_dir2):
    x_train =[]
    y_train =[]
    # Loop through text files in data_dir
    for filename in os.listdir(data_dir1):
    # Read features from text file
        data = np.load(data_dir1+"/"+filename)
        x_train.append(data)
        y_train.append(1.0)
        
    for filename in os.listdir(data_dir2):
    # Read features from text file
        data = np.load(data_dir2+"/"+filename)
        x_train.append(data)
        y_train.append(0.0)  
        
    return x_train, y_train


# ===================== # the GRU model

# size parameters
max_frames=50
max_objects=30
no_of_features=8

# Define model parameters
num_gru_units = 128
num_dense_units = 64
dropout_rate = 0.3

# load the data for training
x_train , y_train =  load_data(f'{HOME}\\CCD_Accident_Sequences\\train',f'{HOME}\\CCD_Normal_Sequences\\train')     # read sequences from DESK
x_train = np.array(x_train)
y_train = np.array(y_train)
x_train = x_train.reshape(3200, -1, max_objects*no_of_features) # flatten each frame to 1D

# validation data
x_val ,y_val =  load_data( f'{HOME}\\CCD_Accident_Sequences\\validate',f'{HOME}\\CCD_Normal_Sequences\\validate'  )     # read sequences from DESK
x_val = np.array(x_val)
y_val = np.array(y_val)
# write the number of validation examples available
x_val = x_val.reshape(800, -1, max_objects*no_of_features) # flatten each frame to 1D

# Define the model
model1 = tf.keras.Sequential()

model1.add(Masking( mask_value=0 , input_shape=( max_frames, max_objects*no_of_features)))

model1.add(GRU(num_gru_units))
model1.add(Dropout(dropout_rate))

# Dense layer for prediction
model1.add(Dense(num_dense_units, activation='relu' ,kernel_regularizer=regularizers.l1_l2(l1=0.01, l2=0.01) ) )
model1.add(Dropout(dropout_rate))

# Output layer with sigmoid activation for probability
model1.add(Dense(1, activation='sigmoid' , kernel_regularizer=regularizers.l1_l2(l1=0.01, l2=0.01) ))
    
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)

# Compile the model
model1.compile(loss='binary_crossentropy', optimizer= optimizer, metrics=['accuracy',f1_m,precision_m,recall_m])

early_stopping = EarlyStopping(monitor='val_accuracy', patience=10)

history = model1.fit(x_train, y_train, epochs=16, batch_size=16, validation_data=(x_val, y_val), callbacks = early_stopping )

# Save the model after training
model1.save('accident_prediction_GRU_model.keras') 

with open('historyGRU.pickle', 'wb') as f:
    pickle.dump(history.history, f)


### plot the evaluation results

In [ ]:
import pickle
import matplotlib.pyplot as plt

# Load the history
with open('historyGRU.pickle', 'rb') as f:
    history = pickle.load(f)

# Plot training & validation accuracy values
plt.plot(history['accuracy'])
plt.plot(history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.ylim(0.4, 0.9)
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()


# Access the values of custom metrics
recall_values = history['recall_m']
precision_values = history['precision_m']
f1_values = history['f1_m']

# Plotting

plt.plot(recall_values, label='Recall')
plt.title('Recall Over Epochs')
plt.ylabel('Recall')
plt.xlabel('Epoch')
plt.ylim(10,50)
plt.legend()
plt.show()

plt.plot(precision_values, label='Precision')
plt.title('Precision over Epochs')
plt.ylabel('precision')
plt.xlabel('Epoch')
plt.ylim(10,50)
plt.legend()
plt.show()

plt.plot(f1_values, label='F1 Score')
plt.title('F1 Score over Epochs')
plt.ylabel('F1_score')
plt.xlabel('Epoch')
plt.ylim(10, 50)
plt.legend()
plt.show()
